In [1]:
import warnings
import pandas as pd
import numpy as np
from astropy.io.votable import parse_single_table, parse
from astropy.io.votable import from_table
from gevtev import gev_tev_data 


warnings.filterwarnings("ignore")

_epsilon = 1e-1

s = "data/simbad-plsr.xml"
votable1 = parse(s)
votable = parse_single_table(s)
table = votable.to_table()
table_plsr = table['RA_d','DEC_d', 'MAIN_ID']
pd_plsr = table_plsr.to_pandas() 
pd_plsr['class'] = 'plsr'

s = "data/simbad-be.xml"
votable1 = parse(s)
votable = parse_single_table(s)
table = votable.to_table()
table_be = table['RA_d','DEC_d', 'MAIN_ID']
pd_be = table_be.to_pandas()
pd_be['class'] = 'be'

frames_simbda = [pd_be, pd_plsr]
pd_simbad = pd.concat(frames_simbda)

In [2]:
common_data, only_tev_data, only_gev_data = gev_tev_data()

In [3]:
def common_gevtev_simbad(gevtev, simbad, epsilon):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a dictionary with corresponding values
    
    gevtev - a table of GeV and TeV 
    simbad - a table of Simbad 
    epsilon(double) - distance accepted as equivalence
    """
    d = {}
    classes = {}
    pos_ra_gevtev = gevtev['pos_ra']
    pos_dec_gevtev = gevtev['pos_dec']
    pos_ra_simbad = pd_simbad.as_matrix(columns=['RA_d'])
    pos_dec_simbad = pd_simbad.as_matrix(columns=['DEC_d'])
    class_simbad = pd_simbad.as_matrix(columns=['class'])
    class_gevtev = gevtev['CLASS1']
    for i in range(len(pos_ra_gevtev)):
        for j in range(len(pos_ra_simbad)):
            if ((np.abs(pos_ra_gevtev[i] - pos_ra_simbad[j])/np.abs(pos_ra_gevtev[i]) < epsilon) 
                and (np.abs(pos_dec_gevtev[i] - pos_dec_simbad[j])/np.abs(pos_dec_gevtev[i]) < epsilon)) :
                    try:
                        list(d.keys()).index(i)
                        d[i].append(j)
                    except ValueError:
                        d.update({i : [j]})
    return d

In [4]:
print(common_gevtev_simbad(common_data, pd_simbad, _epsilon))

{4: [189, 191, 192, 193, 195, 196], 5: [281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 308, 323, 633, 634, 635, 636, 637, 638], 6: [299, 300, 301, 302, 304, 305, 328, 669, 670, 671, 673, 674, 675, 676, 678], 7: [165, 166, 169, 307, 338], 8: [320, 538, 943]}


In [5]:
print(common_gevtev_simbad(only_gev_data, pd_simbad, _epsilon))

{47: [485], 80: [455], 82: [455], 85: [67], 86: [139], 87: [455], 90: [139], 98: [48, 349], 100: [349], 101: [139], 104: [139], 105: [232], 108: [232], 114: [32, 59], 115: [450], 120: [710], 122: [254, 257, 258, 266, 267, 268, 269, 272, 274, 275, 276, 277, 278, 279, 280], 123: [614], 127: [614], 141: [919], 142: [33], 147: [784], 149: [261, 264, 316], 152: [919], 158: [615], 173: [34, 35, 36], 174: [34, 35, 36, 49], 177: [34, 35, 36, 49, 60, 360], 180: [35, 36, 49, 60, 360], 181: [35, 36, 49], 188: [36, 37, 49, 60], 190: [440], 192: [60, 360], 195: [900], 196: [37], 200: [608], 202: [900], 204: [608], 206: [12], 208: [12], 218: [12], 227: [482], 230: [607], 241: [61], 244: [229, 607], 252: [13, 39, 51, 54, 352], 253: [13, 39, 54, 55, 352, 798], 257: [13, 39], 258: [601, 607], 260: [229, 606, 607], 263: [606, 607], 266: [39, 51, 53, 54, 55, 61, 62, 63, 352, 797, 798, 799, 800], 267: [75], 272: [39, 51, 53, 54, 55, 61, 62, 63, 326, 352, 797, 798, 799, 800], 274: [39, 51, 53, 54, 55, 61, 

In [6]:
print(common_gevtev_simbad(only_tev_data, pd_simbad, _epsilon))

{8: [61], 23: [198, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 315, 335, 342, 343, 344, 345, 346, 583, 586, 587, 588, 593, 685, 692, 693, 694, 695, 696, 697, 698, 699, 700, 794, 795], 26: [91, 92, 94, 419, 791], 32: [381], 34: [189, 191, 192, 193, 194, 195, 196, 337, 572], 39: [199, 217, 218, 219, 220, 221, 222, 223, 224, 226, 227, 230, 306, 579, 595, 596, 597, 604, 610, 616, 617, 732, 917], 40: [199, 217, 218, 219, 220, 221, 222, 223, 224, 226, 227, 230, 244, 306, 579, 595, 596, 597, 604, 610, 616, 617, 732, 917], 41: [199, 217, 218, 219, 220, 221, 222, 223, 224, 226, 227, 230, 244, 306, 595, 596, 597, 604, 616, 617, 732, 917], 42: [199, 217, 218, 219, 220, 221, 222, 223, 224, 226, 227, 230, 244, 306, 595, 596, 597, 604, 616, 617, 732, 917], 48: [500], 50: [500, 897], 55: [281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 308, 323, 633, 634, 635, 636, 637, 638], 58: [286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 29